Openpose crop이 이미 적용된 csv, pkl에 대해 정규화를 적용하는 코드,
video_crop_and_csv_merged에 적용되어 있어 이후엔 굳이 실행 안해도 된다

In [ ]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from tqdm import tqdm

# ─────────────────────────────
# 1. 설정
ROOT = Path(r"D:\golfDataset\dataset")  # dataset 루트
SPLITS = ["train", "test"]             # 정규화 대상 split
KP = ["Nose","Neck","RShoulder","RElbow","RWrist","LShoulder","LElbow","LWrist",
      "MidHip","RHip","RKnee","RAnkle","LHip","LKnee","LAnkle","REye","LEye",
      "REar","LEar","LBigToe","LSmallToe","LHeel","RBigToe","RSmallToe","RHeel"]
COLS = [f"{k}_{c}" for k in KP for c in ("x", "y", "c")]

# ─────────────────────────────
# 2. CSV 정규화 함수

def normalize_pose(row):
    arr = row.values.reshape(-1, 3).copy()  # (25, 3)
    if np.isnan(arr).all():
        return row
    midhip = arr[8, :2]
    arr[:, :2] -= midhip
    std = np.std(arr[:, :2])
    if std > 1e-5:
        arr[:, :2] /= std
    return pd.Series(arr.flatten(), index=row.index)

# ─────────────────────────────
# 3. PKL 정규화 함수

def normalize_pose_array(arr):
    for i in range(arr.shape[0]):
        frame = arr[i]
        if np.isnan(frame).all():
            continue
        center = frame[8, :2]
        frame[:, :2] -= center
        std = np.std(frame[:, :2])
        if std > 1e-5:
            frame[:, :2] /= std
    return arr

def normalize_pkl(pkl_path):
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)
    for sample in data['annotations']:
        for person in sample['keypoint']:
            person[:] = normalize_pose_array(person)
    with open(pkl_path, 'wb') as f:
        pickle.dump(data, f)

# ─────────────────────────────
# 4. 실행 루프

for split in SPLITS:
    print(f"\n📁 Split: {split.upper()}")

    # --- CSV 정규화 ---
    kp_root = ROOT / split
    for folder in ["balanced_true", "false"]:
        kp_dir = kp_root / folder / "crop_keypoint"
        if not kp_dir.exists(): continue
        for csv_path in tqdm(sorted(kp_dir.glob("*_crop.csv")), desc=f"[{split}] {folder} CSV"):
            df = pd.read_csv(csv_path)
            df = df[COLS]  # 열 방어
            df_norm = df.apply(normalize_pose, axis=1)
            df_norm.to_csv(csv_path, index=False)

    # --- PKL 정규화 ---
    pkl_dir = kp_root / "crop_pkl"
    for pkl_path in sorted(pkl_dir.glob("skeleton_dataset_*.pkl")):
        print(f"→ PKL 정규화: {pkl_path.name}")
        normalize_pkl(pkl_path)
